In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
import torchvision

In [2]:
# MNIST dataset
dataset = datasets.MNIST(root='./data',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=100, 
                                          shuffle=True)

In [3]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [4]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, h_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(h_dim, z_dim*2))  # 2 for mean and variance.
        
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, image_size),
            nn.Sigmoid())
    
    def reparametrize(self, mu, log_var):
        """"z = mean + eps * sigma where eps is sampled from N(0, 1)."""
        eps = to_var(torch.randn(mu.size(0), mu.size(1)))
        z = mu + eps * torch.exp(log_var/2)    # 2 for convert var to std
        return z
                     
    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = torch.chunk(h, 2, dim=1)  # mean and log variance.
        z = self.reparametrize(mu, log_var)
        out = self.decoder(z)
        return out, mu, log_var
    
    def sample(self, z):
        return self.decoder(z)

In [5]:
vae = VAE()

if torch.cuda.is_available():
    vae.cuda()

In [7]:
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
iter_per_epoch = len(data_loader)
data_iter = iter(data_loader)

# fixed inputs for debugging
fixed_z = to_var(torch.randn(100, 20))
fixed_x, _ = next(data_iter)
torchvision.utils.save_image(fixed_x.cpu(), './data/real_images.png')
fixed_x = to_var(fixed_x.view(fixed_x.size(0), -1))

In [8]:
for epoch in range(50):
    for i, (images, _) in enumerate(data_loader):
        
        images = to_var(images.view(images.size(0), -1))
        out, mu, log_var = vae(images)
        
        # Compute reconstruction loss and kl divergence
        # For kl_divergence, see Appendix B in the paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(out, images, size_average=False)
        kl_divergence = torch.sum(0.5 * (mu**2 + torch.exp(log_var) - log_var -1))
        
        # Backprop + Optimize
        total_loss = reconst_loss + kl_divergence
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print ("Epoch[%d/%d], Step [%d/%d], Total Loss: %.4f, "
                   "Reconst Loss: %.4f, KL Div: %.7f" 
                   %(epoch+1, 50, i+1, iter_per_epoch, total_loss.data[0], 
                     reconst_loss.data[0], kl_divergence.data[0]))
    
    # Save the reconstructed images
    reconst_images, _, _ = vae(fixed_x)
    reconst_images = reconst_images.view(reconst_images.size(0), 1, 28, 28)
    torchvision.utils.save_image(reconst_images.data.cpu(), 
        './data/reconst_images_%d.png' %(epoch+1))

Epoch[1/50], Step [1/600], Total Loss: 54854.3477, Reconst Loss: 54842.3672, KL Div: 11.9787931
Epoch[1/50], Step [101/600], Total Loss: 18328.3574, Reconst Loss: 17132.2246, KL Div: 1196.1320801
Epoch[1/50], Step [201/600], Total Loss: 15042.8262, Reconst Loss: 13538.5996, KL Div: 1504.2260742
Epoch[1/50], Step [301/600], Total Loss: 13655.8027, Reconst Loss: 11799.6631, KL Div: 1856.1394043
Epoch[1/50], Step [401/600], Total Loss: 13582.8047, Reconst Loss: 11577.8877, KL Div: 2004.9174805
Epoch[1/50], Step [501/600], Total Loss: 12176.3105, Reconst Loss: 10137.9766, KL Div: 2038.3339844
Epoch[2/50], Step [1/600], Total Loss: 12523.6182, Reconst Loss: 10226.7598, KL Div: 2296.8586426
Epoch[2/50], Step [101/600], Total Loss: 11784.9941, Reconst Loss: 9547.3330, KL Div: 2237.6613770
Epoch[2/50], Step [201/600], Total Loss: 11903.9521, Reconst Loss: 9574.9248, KL Div: 2329.0275879
Epoch[2/50], Step [301/600], Total Loss: 11672.7500, Reconst Loss: 9377.4121, KL Div: 2295.3376465
Epoch[2/5

KeyboardInterrupt: 